In [224]:
import pandas as pd
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [225]:
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize
import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import os

In [226]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

%matplotlib inline
import os

In [227]:
# Remplace ces imports par les transformations spécifiques à ton problème
# Assure-toi d'avoir les transformations appropriées pour le redimensionnement, la normalisation, etc.
# Vérifie également le nombre de classes dans ton ensemble de données

from torchvision.datasets import ImageFolder

# Définis le chemin vers le répertoire racine de ton ensemble de données extrait
dataset_path = 'flower_images/'

# Définis les transformations à appliquer aux images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Ajuste la taille en fonction de tes besoins
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Ajuste la normalisation
])

# Crée l'ensemble de données en utilisant ImageFolder
dataset = ImageFolder(root=dataset_path, transform=transform)

In [228]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [229]:
import torch.nn as nn
import torch.nn.functional as F
#INPUT => CONV => RELU => FC => SOFTMAX
class CNN(nn.Module):
    def __init__(self,kernel_size,out_channel1,out_channel2):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=out_channel1,
                kernel_size=kernel_size,
                stride=1,
                padding=2,
            ),  
            nn.AdaptiveMaxPool2d(2),
            nn.ReLU(),
        )   
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channel1, out_channel2, kernel_size, 1, 2),
            nn.AdaptiveMaxPool2d(2),
            nn.ReLU(),
        )
        self.dropout = nn.Dropout()
        # Fully connected layer, output 10 classes
        self.out = nn.Linear(out_channel2, 100352)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # Take the average over spatial dimensions
        x = x.mean(dim=[2, 3])
        x = self.dropout(x)

        output = self.out(x)
        return output, x

In [230]:
# Définir la fonction calculate_accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    accuracy = correct / total
    return accuracy

In [231]:
def calculate_accuracy_on_testing_datatset(model,test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:  # Assuming you have a DataLoader for the testing dataset
            outputs = model(inputs)
            _, predicted = torch.max(outputs[0], 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        return correct / total


In [232]:
from torch.autograd import Variable
run_loss = 0

def train(num_epochs, cnn, train_dataloader, optimizer,loss_func,tab_accuracy,test_dataloader):
  global run_loss  # Déclarer la variable globale run_loss
  
  total_step = len(train_dataloader)

  # Boucle d'entraînement
  for epoch in range(num_epochs):
    cnn.train()  # Mettez le modèle en mode d'entraînement
    for i, (images, labels) in enumerate(train_dataloader):
      # gives batch data, normalize x when iterate train_loader
      b_x = Variable(images)   # batch x
      b_y = Variable(labels)   # batch y

      output = cnn(b_x)[0]
      loss = loss_func(output, b_y)

      # clear gradients for this training step
      optimizer.zero_grad()

      # backpropagation, compute gradients
      loss.backward()
      run_loss += loss.item()

      # apply gradients
      optimizer.step()
      
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, accuracy: {}'
              .format(epoch + 1, num_epochs, i + 1, total_step, run_loss / 100,calculate_accuracy(output, b_y)))
      tab_accuracy.append(calculate_accuracy(output, b_y))
      run_loss = 0.0  # Réinitialiser run_loss après chaque batch
      test_accuracy = calculate_accuracy_on_testing_datatset(cnn,test_dataloader)
  print(f'the final accuracy on the testing dataset is {test_accuracy}')
  return test_accuracy
# Assurez-vous d'appeler la fonction train avec les arguments nécessaires

In [233]:
from torch import optim


def HyperParametersChoice(epochs, learning_rate, batch_size,kernel_size,out_channel1,out_channel2):
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    model = CNN(kernel_size,out_channel1,out_channel2)
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)
    tab_accuracy = []
    loss_func = nn.CrossEntropyLoss()   
    train(epochs,model, train_dataloader, optimizer,loss_func,tab_accuracy,test_dataloader)
    return 


In [234]:
HyperParametersChoice(5,0.01,64,5,32,64)#lr 0,001, nombre de filtre 32 ,nombre d'unite dans la couche dense 128,optimizer adam

Epoch [1/5], Step [1/63], Loss: 0.1146, accuracy: 0.0
Epoch [1/5], Step [2/63], Loss: 0.1064, accuracy: 0.03125
Epoch [1/5], Step [3/63], Loss: 0.0870, accuracy: 0.171875
Epoch [1/5], Step [4/63], Loss: 0.0572, accuracy: 0.1875
Epoch [1/5], Step [5/63], Loss: 0.0517, accuracy: 0.171875
Epoch [1/5], Step [6/63], Loss: 0.0365, accuracy: 0.21875
Epoch [1/5], Step [7/63], Loss: 0.0426, accuracy: 0.203125
Epoch [1/5], Step [8/63], Loss: 0.0415, accuracy: 0.203125
Epoch [1/5], Step [9/63], Loss: 0.0444, accuracy: 0.171875
Epoch [1/5], Step [10/63], Loss: 0.0379, accuracy: 0.171875
Epoch [1/5], Step [11/63], Loss: 0.0278, accuracy: 0.234375
Epoch [1/5], Step [12/63], Loss: 0.0336, accuracy: 0.171875
Epoch [1/5], Step [13/63], Loss: 0.0276, accuracy: 0.234375
Epoch [1/5], Step [14/63], Loss: 0.0256, accuracy: 0.15625
Epoch [1/5], Step [15/63], Loss: 0.0237, accuracy: 0.265625
Epoch [1/5], Step [16/63], Loss: 0.0228, accuracy: 0.21875
Epoch [1/5], Step [17/63], Loss: 0.0255, accuracy: 0.15625
E

In [241]:
def finding_hyperparameters():
    tab_accuracy=[]
    for learning_rate in range(1,4):
        for kernelsize in range(3,6):
            for outchannel1 in range(1,3):
                for outchannel2 in range(1,3):
                    accuracy = HyperParametersChoice(5,0.01*learning_rate,64,kernelsize,32*outchannel1,32*outchannel2)
                    tab_accuracy.append({'learning rate' : learning_rate,
                                         'kernel size' : kernelsize,
                                         'outchannel1' : outchannel1,
                                         'outchannel2': outchannel2})
                    print(f'The accuracy for Learning rate : {learning_rate}, kernel size : {kernelsize}, out channel for the first layer : {outchannel1}, out channel for the second layer : {outchannel2} is {accuracy}')
    return tab_accuracy
                    

In [242]:
tab_accuracy = finding_hyperparameters()

Epoch [1/5], Step [1/63], Loss: 0.1150, accuracy: 0.0
Epoch [1/5], Step [2/63], Loss: 0.1103, accuracy: 0.015625
Epoch [1/5], Step [3/63], Loss: 0.1042, accuracy: 0.09375
Epoch [1/5], Step [4/63], Loss: 0.0947, accuracy: 0.1875
Epoch [1/5], Step [5/63], Loss: 0.0863, accuracy: 0.15625
Epoch [1/5], Step [6/63], Loss: 0.0770, accuracy: 0.171875
Epoch [1/5], Step [7/63], Loss: 0.0575, accuracy: 0.3125
Epoch [1/5], Step [8/63], Loss: 0.0648, accuracy: 0.15625
Epoch [1/5], Step [9/63], Loss: 0.0385, accuracy: 0.28125
Epoch [1/5], Step [10/63], Loss: 0.0428, accuracy: 0.21875
Epoch [1/5], Step [11/63], Loss: 0.0412, accuracy: 0.125
Epoch [1/5], Step [12/63], Loss: 0.0367, accuracy: 0.21875
Epoch [1/5], Step [13/63], Loss: 0.0328, accuracy: 0.28125
Epoch [1/5], Step [14/63], Loss: 0.0330, accuracy: 0.203125
Epoch [1/5], Step [15/63], Loss: 0.0319, accuracy: 0.21875
Epoch [1/5], Step [16/63], Loss: 0.0319, accuracy: 0.125
Epoch [1/5], Step [17/63], Loss: 0.0286, accuracy: 0.15625
Epoch [1/5], 